# Download and look into the data

## Project description

The telecom operator Interconnect would like to be able to forecast their churn of clients. If it's discovered that a user is planning to leave, they will be offered promotional codes and special plan options. Interconnect's marketing team has collected some of their clientele's personal data, including information about their plans and contracts.

**Interconnect's services**

Interconnect mainly provides two types of services:

1. Landline communication. The telephone can be connected to several lines simultaneously.
2. Internet. The network can be set up via a telephone line (DSL, *digital subscriber line*) or through a fiber optic cable.

Some other services the company provides include:

- Internet security: antivirus software (*DeviceProtection*) and a malicious website blocker (*OnlineSecurity*)
- A dedicated technical support line (*TechSupport*)
- Cloud file storage and data backup (*OnlineBackup*)
- TV streaming (*StreamingTV*) and a movie directory (*StreamingMovies*)

The clients can choose either a monthly payment or sign a 1- or 2-year contract. They can use various payment methods and receive an electronic invoice after a transaction.

### Import needed libraries

In [32]:
# Data tools
import pandas as pd
import numpy as np
#from pandas_profiling import ProfileReport

# others
import time
from functools import reduce

# Graphics and display
from IPython.core.interactiveshell import InteractiveShell
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

# Ml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
!pip install lightgbm 
import lightgbm as lgb
!pip install catboost
import catboost as cb

# Statistics
from scipy import stats

print('Project libraries has been successfully been imported!')

1.0.3  Set environment variables

In [5]:
# set to display all output not only print() or last output
InteractiveShell.ast_node_interactivity = "all"      

1.1  Open the file and look into the data.

In [6]:
try:
    contract = pd.read_csv('contract.csv')
    internet = pd.read_csv('internet.csv')
    personal = pd.read_csv('personal.csv')
    phone = pd.read_csv('phone.csv')
    
except:
    contract = pd.read_csv('/datasets/contract.csv')
    internet = pd.read_csv('/datasets/internet.csv')
    personal = pd.read_csv('/datasets/personal.csv')
    phone = pd.read_csv('/datasets/phone.csv')
    
print('Data has been read correctly!')

Data has been read correctly!


##  Data describe notes

#### Special function for data describe

In [7]:
# Checking 0's
def zero_check(df):
    for i in df:
        print(i,len(df[df[i]==0]))
        
# function to determine if columns in file have null values        
def get_percent_of_na(df, num):
    count = 0
    df = df.copy()
    s = (df.isna().sum() / df.shape[0])
    for column, percent in zip(s.index, s.values):
        num_of_nulls = df[column].isna().sum()
        if num_of_nulls == 0:
            continue
        else:
            count += 1
        print('{} has {} nulls, which is {:.{}%} percent of Nulls'.format(column, num_of_nulls, percent, num))
    if count != 0:
        print("\033[1m" + 'There are {} columns with NA.'.format(count) + "\033[0m")
    else:
        print()
        print("\033[1m" + 'There are no columns with NA.' + "\033[0m")       
        
# function to display general information about the dataset
def general_info(df):
    print("\033[1m" + "\033[0m")
    display(pd.concat([df.dtypes, df.count(),df.isna().sum(),df.isna().sum()*100/len(df)], keys=['type','count','na','na%'],
                      axis=1))
    print()
    print("\033[1m" + 'Head:')  
    display(df.head())
    print()
    print("\033[1m" + 'Tail:')
    display(df.tail())
    print()
    print("\033[1m" + 'Info:')
    print()
    display(df.info())
    print()
    print("\033[1m" + 'Describe:')
    print()
    display(df.describe())
    print()
    print("\033[1m" + 'Describe include: all :')
    print()
    display(df.describe(include='all'))
    print()
    print("\033[1m" + 'nulls in the columns:')
    print()
    display(get_percent_of_na(df, 4))  # check this out
    print()
    print("\033[1m" + 'Zeros in the columns:') 
    print()
    display(zero_check(df))
    print()
    print("\033[1m" + 'Shape:', df.shape)
    print()
    print()
    print('Duplicated:',"\033[1m" + 'We have {} duplicated rows\n'.format(df.duplicated().sum()) + "\033[0m")
    print()
    print("\033[1m" + 'Dtypes:')  
    display(df.dtypes)
    print()

In [8]:
# check if there are id that return more than once
def each_id_appears_once(df):
    if len(df['customerID'].unique()) == len(df['customerID']):
        return print("each id appears only once")
    else:
        return print("each id appears more then once")  

In [9]:
def is_all_id_in_sub_df_are_within_main_data_base(sub_df, main_df=contract):
    list_main = main_df['customerID'].to_list()
    list_sub = sub_df['customerID'].to_list()
    check =  all(item in list_main for item in list_sub)
    if check is True:
        print("list_main contains all elements of the list_sub")    
    else :
        print("No, List1 doesn't have all elements of the List2.")

In [10]:
print('information about the contract dataset:')
general_info(contract)

information about the contract dataset:



,type,count,na,na%
customerID,object,7043,0,0.0
BeginDate,object,7043,0,0.0
EndDate,object,7043,0,0.0
Type,object,7043,0,0.0
PaperlessBilling,object,7043,0,0.0
PaymentMethod,object,7043,0,0.0
MonthlyCharges,float64,7043,0,0.0
TotalCharges,object,7043,0,0.0



Head:


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65



Tail:


...

None


Zeros in the columns:

customerID 0
BeginDate 0
EndDate 0
Type 0
PaperlessBilling 0
PaymentMethod 0
MonthlyCharges 0
TotalCharges 0


None


Shape: (7043, 8)


Duplicated: We have 0 duplicated rows


Dtypes:


customerID           object
BeginDate            object
EndDate              object
Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
dtype: object

In [11]:
each_id_appears_once(contract)

each id appears only once


In [12]:
contract['PaymentMethod'].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

In [13]:
contract['Type'].unique()

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [14]:
contract['PaperlessBilling'].unique()

array(['Yes', 'No'], dtype=object)

In [15]:
contract['EndDate'].unique()

array(['No', '2019-12-01 00:00:00', '2019-11-01 00:00:00',
       '2019-10-01 00:00:00', '2020-01-01 00:00:00'], dtype=object)

In [16]:
contract['BeginDate'].unique()

array(['2020-01-01', '2017-04-01', '2019-10-01', '2016-05-01',
       '2019-09-01', '2019-03-01', '2018-04-01', '2019-04-01',
       '2017-07-01', '2014-12-01', '2019-01-01', '2018-10-01',
       '2015-04-01', '2015-09-01', '2018-01-01', '2014-05-01',
       '2015-10-01', '2014-03-01', '2018-05-01', '2019-11-01',
       '2019-02-01', '2016-01-01', '2017-08-01', '2015-12-01',
       '2014-02-01', '2018-06-01', '2019-12-01', '2017-11-01',
       '2019-06-01', '2016-04-01', '2017-02-01', '2018-12-01',
       '2014-04-01', '2018-09-01', '2014-11-01', '2016-07-01',
       '2015-02-01', '2018-07-01', '2014-08-01', '2016-03-01',
       '2018-08-01', '2014-10-01', '2015-06-01', '2016-08-01',
       '2019-05-01', '2017-03-01', '2016-02-01', '2017-09-01',
       '2014-09-01', '2017-12-01', '2016-12-01', '2017-06-01',
       '2015-05-01', '2016-10-01', '2016-09-01', '2019-08-01',
       '2019-07-01', '2017-05-01', '2017-10-01', '2014-07-01',
       '2018-03-01', '2015-01-01', '2018-11-01', '2015-

In [18]:
contract['BeginDate'].value_counts()

2014-02-01    366
2019-10-01    237
2019-11-01    237
2019-09-01    237
2020-01-01    233
             ... 
2020-02-01     11
2014-01-01      7
2013-10-01      3
2013-12-01      3
2013-11-01      2
Name: BeginDate, Length: 77, dtype: int64

In [21]:
frac_No_for_ballance = len(contract[contract['EndDate'] == 'No']) / len(contract['EndDate'])
print('The "No" precent in the EndDate column (target) is: {:.0%}'.format(frac_No_for_ballance))

The "No" precent in the EndDate column (target) is: 73%


### Notes about the contract data base:
In the contract we have all the customers id - 7043 total. The data is unballanced (73% not left the program)
No missing values.

Task in the preprocessing:
- Need to correct columns name to underscore lowercase.
- change BeginDate date to date type
- TotalCharges change to float64

Task in the features engineering:
- PaymentMethod, Type, PaperlessBilling with one hot encoding
- Create column based on the EndDate says if customer left or not the service.
- Check seasonality for BeginDate (month, day of the week, hour a day)
- After splitting perform class ballancing


       

In [23]:
print('information about the internet dataset:')
general_info(internet)

information about the internet dataset:



,type,count,na,na%
customerID,object,5517,0,0.0
InternetService,object,5517,0,0.0
OnlineSecurity,object,5517,0,0.0
OnlineBackup,object,5517,0,0.0
DeviceProtection,object,5517,0,0.0
TechSupport,object,5517,0,0.0
StreamingTV,object,5517,0,0.0
StreamingMovies,object,5517,0,0.0



Head:


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No



Tail:


...

None


Zeros in the columns:

customerID 0
InternetService 0
OnlineSecurity 0
OnlineBackup 0
DeviceProtection 0
TechSupport 0
StreamingTV 0
StreamingMovies 0


None


Shape: (5517, 8)


Duplicated: We have 0 duplicated rows


Dtypes:


customerID          object
InternetService     object
OnlineSecurity      object
OnlineBackup        object
DeviceProtection    object
TechSupport         object
StreamingTV         object
StreamingMovies     object
dtype: object

In [22]:
each_id_appears_once(internet)

each id appears only once


In [24]:
# check if the internet databse id are within the contract data base
is_all_id_in_sub_df_are_within_main_data_base(internet)

list_main contains all elements of the list_sub


### Notes about Internet data base:


Here we have information about all users with Internet service. We can immediately see that 7043-5517= 1526 don't have Internet service and therefore they are not appears in this database.

Task in the preprocessing:
- Need to correct columns name to underscore lowercase.

Task in the features engineering:
- merge this data to the contract data that will become the main df
- add to InternetService column a category of - NO. This will suite the rest 1526 users that have no Internet at all. Same all the rest of the columns will receive NO for them.
- Do one hot encoding for all these features


In [25]:
print('information about the personal dataset:')
general_info(personal)

information about the personal dataset:



,type,count,na,na%
customerID,object,7043,0,0.0
gender,object,7043,0,0.0
SeniorCitizen,int64,7043,0,0.0
Partner,object,7043,0,0.0
Dependents,object,7043,0,0.0



Head:


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No



Tail:


...

None


Zeros in the columns:

customerID 0
gender 0
SeniorCitizen 5901
Partner 0
Dependents 0


None


Shape: (7043, 5)


Duplicated: We have 0 duplicated rows


Dtypes:


customerID       object
gender           object
SeniorCitizen     int64
Partner          object
Dependents       object
dtype: object

In [26]:
is_all_id_in_sub_df_are_within_main_data_base(sub_df=personal)

list_main contains all elements of the list_sub


In [27]:
each_id_appears_once(personal)

each id appears only once


### Notes about personal data base:

Here we see the same users as in the contract (main). This is just addition of data so we can simply merge it to the main.


In [28]:
print('information about the phone dataset:')
general_info(phone)

information about the phone dataset:



,type,count,na,na%
customerID,object,6361,0,0.0
MultipleLines,object,6361,0,0.0



Head:


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes



Tail:


...

None


Zeros in the columns:

customerID 0
MultipleLines 0


None


Shape: (6361, 2)


Duplicated: We have 0 duplicated rows


Dtypes:


customerID       object
MultipleLines    object
dtype: object

In [29]:
is_all_id_in_sub_df_are_within_main_data_base(sub_df=phone)

list_main contains all elements of the list_sub


In [30]:
each_id_appears_once(phone)

each id appears only once


### Notes about phone data base:
not all users have a phone line. What we can do here is to merge this database to the main data base and use OHE as so:
- Does it have only one user?
- Does it have multiple users?
If both answers be no this is someone without phone plan

# Proposed work plan

In the telecom project our goal is to develop a model that can tell is user will leave the service.
The steps to achieve that will be:

1. Download the data


2. Explore the data to determine how to treat the data in the preprocessing


3. Perform preprocessing for the data that will include
    - merge the data from all databases to one main dataframe
    - Change column name to consist format
    - convert to desired type 
    
4. Perform EDA to explore the data in depth 
 
 
5. Feature engineering 

     produce features that classification model can handle That don't create data leakage between train and test set.
    
    
6. Optional - Use unsupervised learning to see if there are clusters in the data - It can be interesting to see if the endate that are not "No" match with the clusters. If the answer to this will be yes then after split train and target to do it again for feature generation.
    
    
7. Prepare data for modeling

    - Split for tain & test in ratio of 4:1 in stratified way
    - Preform encodings (OHE and ordinal) as well as scaling since data is unbalanced
    
    
8. Model training

    - For at least 3 different models select best hyperparameters using cross-validation and validate with AUC-ROC.
    - Train models with best hyperparameters on all train set.
    - For sanity check use one dummy model


9. Model testing

    - Test each model with the test set and choose the one producing the best AUC-ROC score. 
    - If the desired story points not achieve then return to step 5 
 
 
10. Conclusions

    Draw conclusions on the model
    
Questions to the team leader
- How come there are only 4 possible contract EndDate?
       '2019-12-01', '2019-11-01', '2019-10-01', '2020-01-01'
- Please refer to my step 6. Is that something that I can do? If not how can I use unsupervised learning as a way to improve my classification model? 
    

# Preprocessing

## Merge data bases

In [33]:
# merge all dataframes

# compile the list of dataframes you want to merge
data_frames = [contract, internet, personal, phone]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['customerID'],
                                            how='outer'), data_frames)


In [35]:
df_merged.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No


In [36]:
get_percent_of_na(df_merged, 4)

InternetService has 1526 nulls, which is 21.6669% percent of Nulls
OnlineSecurity has 1526 nulls, which is 21.6669% percent of Nulls
OnlineBackup has 1526 nulls, which is 21.6669% percent of Nulls
DeviceProtection has 1526 nulls, which is 21.6669% percent of Nulls
TechSupport has 1526 nulls, which is 21.6669% percent of Nulls
StreamingTV has 1526 nulls, which is 21.6669% percent of Nulls
StreamingMovies has 1526 nulls, which is 21.6669% percent of Nulls
MultipleLines has 682 nulls, which is 9.6834% percent of Nulls
There are 8 columns with NA.


## Fill nan

In [53]:
# fill the nan values from internet database
internet_columns = internet.columns.to_list()[1:]
print('The values in each column in original internet data base')
print('---------------------------------------------------------')
print()
for column in internet_columns:
    print(column)
    print(internet[column].unique())
    print()


The values in each column in original internet data base
---------------------------------------------------------

InternetService
['DSL' 'Fiber optic']

OnlineSecurity
['No' 'Yes']

OnlineBackup
['Yes' 'No']

DeviceProtection
['No' 'Yes']

TechSupport
['No' 'Yes']

StreamingTV
['No' 'Yes']

StreamingMovies
['No' 'Yes']



I will fill No in all the nan for these columns 

In [65]:
df_merged[internet_columns] = df_merged[internet_columns].fillna(value='No')

In [66]:
get_percent_of_na(df_merged, 4)

MultipleLines has 682 nulls, which is 9.6834% percent of Nulls
There are 1 columns with NA.


In [67]:
# fill the nan values in MultipleLines column

phone_columns = phone.columns.to_list()[1:]
print('The values in each column in original phone data base')
print('---------------------------------------------------------')
print()
for column in phone_columns:
    print(column)
    print(phone[column].unique())
    print()


The values in each column in original phone data base
---------------------------------------------------------

MultipleLines
['No' 'Yes']



I will replace this with num_of_lines
- 0 for nan
- 1 for 1 line (MultipleLines = No)
- 1 for more than 1 line (MultipleLines = Yes)

In [77]:
df_merged['MultipleLines'] = df_merged['MultipleLines'].replace(
    {'No': 1, 'Yes':2}
).fillna(value=0)

In [78]:
get_percent_of_na(df_merged, 4)


There are no columns with NA.


## Change columns names

In [94]:
column_list = ['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'MultipleLines']

In [95]:
[column.lower() for column in column_list]

['customerid',
 'begindate',
 'enddate',
 'type',
 'paperlessbilling',
 'paymentmethod',
 'monthlycharges',
 'totalcharges',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'gender',
 'seniorcitizen',
 'partner',
 'dependents',
 'multiplelines']

In [96]:
df_merged.columns = ['customer_id',
 'begin_date',
 'end_date',
 'type',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'total_charges',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'multiple_lines']

In [97]:
df_merged.columns

Index(['customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges',
       'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streamingm_ovies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines'],
      dtype='object')

## Convert type

### Date

In [104]:
# to datetime
df_merged['begin_date'] = pd.to_datetime(df_merged['begin_date'], format='%Y.%m.%d')

### Numerical

In [122]:
# convert to numerical
df_merged['total_charges'].value_counts()

0         11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: total_charges, Length: 6531, dtype: int64

replace the ' '  with 0

In [120]:
df_merged['total_charges'] = df_merged['total_charges'].replace(to_replace=' ', value='0')

In [121]:
df_merged['total_charges'].value_counts()

0         11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: total_charges, Length: 6531, dtype: int64

In [123]:
# to numeric
df_merged['total_charges'] = df_merged['total_charges'].astype(float)

### Categorical

In [130]:
column_to_categorical = ['type', 'paperless_billing', 'payment_method', 'internet_service',
 'online_security', 'online_backup','device_protection', 'tech_support', 'streaming_tv',
 'streamingm_ovies', 'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines']

In [135]:
for column in column_to_categorical:
    df_merged[column] = df_merged[column].astype('category')

### create churn feature

In [141]:
def churn_0_1(row):
    end_date = row['end_date']
    if end_date == 'No':
        return 0
    else:
        return 1

In [146]:
df_merged['churn'] = df_merged.apply(churn_0_1, axis=1)

### List features and target

In [147]:
df_merged.columns

Index(['customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges',
       'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streamingm_ovies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines',
       'churn'],
      dtype='object')

In [153]:
features = ['begin_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges',
       'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streamingm_ovies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines']
target = 'churn'

# EDA 

## Univariate EDA

In [246]:
numerical_features_for_affecting_churn_test = [
'monthly_charges', 'total_charges'
]

In [253]:
px.histogram(data_frame=df_merged['monthly_charges'])

In [249]:
px.histogram(data_frame=df_merged['total_charges'])

## Bivariate EDA

Check how categorical features affect by churn rate - target

In [181]:
features_for_affecting_churn_test = [
'type', 'paperless_billing', 'payment_method', 
'internet_service', 'online_security', 'online_backup','device_protection',
'tech_support', 'streaming_tv', 'streamingm_ovies', 'gender', 'senior_citizen',
'partner', 'dependents', 'multiple_lines']

In [226]:
def create_pivot_for_churn(index:str):
    pivot_table_churn = df_merged.pivot_table(
    columns='churn',
    index=index,
    values= 'customer_id',
    aggfunc='count'
    
                     )
    pivot_table_churn[1] = pivot_table_churn[1] / pivot_table_churn[1].sum()
    pivot_table_churn[0] = pivot_table_churn[0] / pivot_table_churn[0].sum()
    return pivot_table_churn

In [227]:
def churn_bar(table):
    px.bar(data_frame=table.T, title=feature).show()

In [228]:
for feature in features_for_affecting_churn_test:
    table = create_pivot_for_churn(feature)
    print(feature)
    churn_bar(table)

type


paperless_billing


payment_method


internet_service


online_security


...

gender


senior_citizen


partner


dependents


multiple_lines


**Categorical features that affect churn**

- type - month to month customers tend to churn more. one year and two year most likely not to churn

- paperless_billing - customers who use paperless_billing tend to churn more then those who aren't

- payment_method - The customers who use electronic check tend to churn more

- internet_service - customers that don't use internet service are less likely to churn while those who use the high end Fiber optic are more likely

An intermediate observation - The more technological oriented the customer the more likely it to churn. Maybe we can see it on the age group - my hypothesis is that senior citizens less likely to churn. We will check that soon.

- online_security - The customers who use online_security service tend to churn less

- tech_support - The customers who use tech_support tend to churn less

- senior_citizen - senior_citizen actually churn more so my hypothesis was wrong

- partner - customers with no partner tend to churn more

- dependents - customers with no dependents tend to churn more

- 


## Multivariate EDA

Check how numerical features affect by churn rate - target

In [243]:
px.scatter(
    data_frame=df_merged.replace(to_replace={0:'No', 1:'Yes'}),
    x='begin_date', y='monthly_charges', color='churn'
)

We can see that monthly charge don't cross 120 (no high outliers) and most churn happen for late join users and to users with high monthly charge.

In [244]:
px.scatter(
    data_frame=df_merged.replace(to_replace={0:'No', 1:'Yes'}),
    x='begin_date', y='total_charges', color='churn'
)

Again we see that most churn happen for the late users. There is probably a trend that we will soon check. 

# Feature engineering

Encoding (OHE, Ordinal), timeseries, 

In [229]:
pivot_type = df_merged.pivot_table(
    columns='churn',
    index='type',
    values= 'customer_id',
    aggfunc='count'
    
                     )

pivot_type[1] = round(pivot_type[1] / pivot_type[1].sum(), 2)
pivot_type[0] = round(pivot_type[0] / pivot_type[0].sum(), 2)

In [230]:
pivot_type

churn,0,1
type,,
Month-to-month,2220,1655
One year,1307,166
Two year,1647,48


In [231]:
px.bar(pivot_type.T)

In [138]:
from pandas_profiling import ProfileReport

In [139]:
profile = ProfileReport(df_merged, title="Pandas Profiling Report")

In [140]:
profile.to_file("your_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [99]:
df_merged_test = df_merged.copy(deep=True)

In [63]:
df_merged_test[internet_columns] = df_merged_test[internet_columns].fillna(value='No')

In [64]:
get_percent_of_na(df_merged_test, 4)

MultipleLines has 682 nulls, which is 9.6834% percent of Nulls
There are 1 columns with NA.


In [44]:
internet.columns.to_list()[1:]

['InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies']

In [46]:
internet.InternetService.unique()

array(['DSL', 'Fiber optic'], dtype=object)